In [1]:
!pip install pydriller
!pip install PyGithub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.4/68.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 35.2 MB/s eta 0:00:00


In [2]:
!pip install Github

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.0 MB/s eta 0:00:00
  Attempting uninstall: charset-normalizer
    Found existing installation: charset-normalizer 3.4.0
    Uninstalling charset-normalizer-3.4.0:
      Successfully uninstalled charset-normalizer-3.4.0
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.10
    Uninstalling aiohttp-3.11.10:
      Successfully uninstalled aiohttp-3.11.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.12 requires aiohttp<4.0.0,>=3.8.3, but you have aiohttp 3.8.1 which is incompatible.


In [3]:
import requests
import pandas as pd
import csv
#from github import Github

In [4]:
token = 'Your_token'

In [5]:
# Can return at max 1000 repos per query
query = 'language:solidity stars:>=10 created:2013-12-01T00:00:00Z..2024-12-31T00:00:00Z&order=asc'

In [6]:
per_page = 100
url = f"https://api.github.com/search/repositories?q={query}&per_page={per_page}"

headers = {
  "Authorization": "Bearer Token"
}
response = requests.get(url)
data = response.json()
print(data)

total_count = data['total_count']
print(f"Total repositories: {total_count}")

{'total_count': 5930, 'incomplete_results': False, 'items': [{'id': 64700934, 'node_id': 'MDEwOlJlcG9zaXRvcnk2NDcwMDkzNA==', 'name': 'openzeppelin-contracts', 'full_name': 'OpenZeppelin/openzeppelin-contracts', 'private': False, 'owner': {'login': 'OpenZeppelin', 'id': 20820676, 'node_id': 'MDEyOk9yZ2FuaXphdGlvbjIwODIwNjc2', 'avatar_url': 'https://avatars.githubusercontent.com/u/20820676?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/OpenZeppelin', 'html_url': 'https://github.com/OpenZeppelin', 'followers_url': 'https://api.github.com/users/OpenZeppelin/followers', 'following_url': 'https://api.github.com/users/OpenZeppelin/following{/other_user}', 'gists_url': 'https://api.github.com/users/OpenZeppelin/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/OpenZeppelin/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/OpenZeppelin/subscriptions', 'organizations_url': 'https://api.github.com/users/OpenZeppelin/orgs', 'repos_url': 'https:

In [7]:
repositories = data['items']

In [8]:
rows = list()

In [9]:
repositories=[]
page = 1
while len(rows) < total_count:
    url = f"https://api.github.com/search/repositories?q={query}&per_page={per_page}&page={page}"
    response = requests.get(url, headers=headers)
    data = response.json()
    #print(data)

    try:
        is_incomplete_result=data['incomplete_results']
        if is_incomplete_result:
          print('incomplete')
        else:
          print(len(rows))

          repositories = data['items']
    except KeyError:
        print('error')
        break

    for repo in repositories:
        #print(repo['name'])
        repo_name = repo['name']
        commits_url = repo['commits_url'].split("{")[0]
        stars = repo['stargazers_count']
        repo_url = repo['html_url']

        # Get number of commitshttps://github.com/consenlabs/tokenlon-con…
        try:
            commits_response = requests.get(commits_url, headers=headers)
            commits_data = commits_response.json()
            num_commits = len(commits_data)
        except:
            num_commits = 0
        row_to_append = {
                        "name": repo_name,
                        "num_commit": num_commits,
                        "stars": stars,
                        "url": repo_url,
                        }

        rows.append(row_to_append)

        if len(rows) >= total_count:
            break

    page += 1
    if page==10:
      break;

0
100
200
300
400
500
600
700
800


In [10]:
fieldnames=["name", "num_commit", "stars", "url"]
with open('study_context_emse.csv', 'a', newline='') as file:
  writer = csv.DictWriter(file, fieldnames=fieldnames)

  if file.tell() == 0:
    writer.writeheader()

  writer.writerows(rows)

In [11]:
df = pd.read_csv("study_context_emse.csv")
df_no_duplicates = df.drop_duplicates()
df_no_duplicates.shape

(900, 4)

In [12]:
df_no_duplicates = df.drop_duplicates()
df_no_duplicates.shape

(900, 4)

df.to_csv('solidity_repos_filtered.csv', index=False)